# [실습] 다양한 Graph 구조

그동안 배운 요소들을 바탕으로, 이번에는 기존의 Graph 구조를 보다 확장시켜 보겠습니다.   

간단한 Router 구조,

하나의 출발점에서 여러 개로 분리되는 Parallel Calling 이후에 결과를 합치는 Map Reduce 방식,

생성자와 평가자의 구조를 반복하는 Evaluator-Optimizer 방식을 구현해 보겠습니다.



In [ ]:
!pip install langgraph langchain langchain_google_genai langchain_community

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = 'AIxxx'

from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    rate_limiter=rate_limiter,
    # temperature
    # max_tokens

    thinking_budget = 500  # 추론(Reasoning) 토큰 길이 제한
)

## 1. Router
라우터는 State의 값을 참고하여, 목적에 따라 서로 다른 노드로 전달하는 방식을 의미합니다.   
주로 사용자의 입력을 분류하여 서로 다른 작업을 연결하는 의도 분류(Intent Classfication)에서 활용됩니다.

In [ ]:
from typing_extensions import TypedDict, Annotated, Literal, List
from pydantic import BaseModel, Field
from langgraph.graph.message import add_messages

class Recipe(BaseModel):
    name: str = Field(..., description="음식 이름")
    difficulty: str = Field(..., description="만들기의 난이도")
    origin: str = Field(..., description="원산지")
    ingredients: List[str] = Field(..., description="재료 목록")
    instructions: List[str] = Field(..., description="조리법")
    taste: List[str] = Field(..., description="맛에 대한 한 마디의 묘사!")

class Movie(BaseModel):
    name: str = Field(..., description="영화 이름")
    director: str = Field(..., description="감독명")
    actor: List[str] = Field(..., description="주연 배우: 최대 3명까지")
    recommendation: str = Field(..., description="추천하는 이유!")


class State(TypedDict):
    query: str
    classification: str
    recipe: Recipe
    movie:Movie
    advice : Literal['네!', '아니오.']
    # Literal: 범위가 특정 값으로 한정되는 경우
    answer: str



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
import random

def recommend_recipe(state):
    prompt = ChatPromptTemplate([
    ('system','당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.'),
    ('user','''{query}''')
    ])

    recipe_chain = prompt | llm.with_structured_output(Recipe)

    return {'recipe':recipe_chain.invoke(state)}
    # query --> query

def recommend_movie(state):
    prompt = ChatPromptTemplate([
    ('system','당신은 고전 영화의 전문가입니다.'),
    ('user','''{query}''')
    ])

    movie_chain = prompt | llm.with_structured_output(Movie)

    return {'movie':movie_chain.invoke(state)}
    # query --> query


def talk(state):
    return {'answer':llm.invoke(state['query']).content}


def counsel(state):
    if random.random()>=0.5:
        return {'advice':'네!'}
    else:
        return {'advice':'아니오.'}



In [ ]:
def route(state):

    prompt = ChatPromptTemplate(
        [('system', '''당신의 역할은 사용자의 질문에 대답할 사람을 선택하는 것입니다.
1) 음식 관련 질문: 'FOOD'만 출력하세요.
2) 영화 관련 질문: 'MOVIE'만 출력하세요.
3) 고민 상담: 'COUNSEL'만 출력하세요.
4) 그 외의 대화: 'TALK'만 출력하세요.
             '''),
             ('user','User Query: {query}')
        ]
    )
    # Structured_Output으로 만들 수도 있습니다!

    route_chain = prompt | llm

    return {"classification": route_chain.invoke(state).content}
    # query --> query


def route_decision(state):
    # Exact Match 대신 조금 안정적인 조건식

    if "FOOD" in state["classification"]:
        return "recommend_recipe"
    elif "MOVIE" in state["classification"]:
        return "recommend_movie"
    elif "TALK" in state["classification"]:
        return "talk"
    else:
        return "counsel"

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(State)

builder.add_node('recommend_movie', recommend_movie)
builder.add_node('recommend_recipe', recommend_recipe)
builder.add_node('counsel', counsel)
builder.add_node('talk', talk)
builder.add_node('route', route)

builder.add_edge(START, 'route')
builder.add_conditional_edges('route', route_decision,
                              {'recommend_movie':'recommend_movie',
                               'recommend_recipe':'recommend_recipe',
                               'counsel':'counsel',
                               'talk':'talk'})

builder.add_edge('recommend_movie', END)
builder.add_edge('recommend_recipe', END)
builder.add_edge('counsel', END)
builder.add_edge('talk', END)


In [ ]:
graph = builder.compile()
graph

In [ ]:
query = '2월에 어울리는 한국영화 추천해줘.'

result = graph.invoke({'query':query})
result

In [ ]:
query = '연두부로 만들 수 있는 파인 다이닝 메뉴 추천해주세요'

result = graph.invoke({'query':query})
result

In [ ]:
query = 'MoE 구조에 대해 5문장으로 설명해주세요.'

result = graph.invoke({'query':query})
result

## 2. Map-Reduce

위에서는 분류 후에 1번의 LLM을 호출했는데요.   
각자 실행하고 합치는 구조도 만들 수 있습니다.   

대표적인 작업인 리포트 작성 구조를 보겠습니다.   
최초의 LLM이 주제에 대한 섹션을 먼저 구성하고, 섹션별 리포트를 각각의 LLM이 작성하는 방식입니다.

In [ ]:
# 전체 섹션의 구획: Contents (Chapter List)
# Chapter: name, outline
class Chapter(BaseModel):
    name: str = Field(description="챕터의 이름")
    outline: str = Field(description="챕터의 주요 내용, 1문장 길이로")


class Contents(BaseModel):
    contents: List[Chapter] = Field(description="전체 리포트의 섹션 구성")


planner = llm.with_structured_output(Contents)

In [ ]:
example = planner.invoke("LLM의 발전 과정에 대한 보고서 구획을 작성해 주세요.")
example.contents

그래프에서 사용할 State를 정의합니다.   

이번에는 중간 Writer LLM이 사용할 State를 별도로 만들어 보겠습니다.   
이렇게 구성하면 최종 State에서 필요한 부분만 저장할 수 있습니다.

In [ ]:
import operator

# reducer 구조: operator.add
# 단순 + 연산 구조 (리스트의 + 연산이므로 append)

class State(TypedDict):
    topic: str
    contents: list[Chapter]
    completed_sections: Annotated[list, operator.add]
    final_report: str


# 섹션 Writer가 사용할 State
class SubState(TypedDict):
    chapter: Chapter
    completed_sections: Annotated[list, operator.add]



섹션을 생성하는 노드를 구성합니다.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

def orchestrator(state: State):

    prompt = ChatPromptTemplate([
        ('system', "주제에 대한 전문가 수준의 깊이 있는 한국어 보고서를 쓰려고 합니다. 보고서의 섹션 구성과, 각 섹션의 간단한 설명을 작성해 주세요."),
        ('user', "주제: {topic}")
    ])
    chain = prompt | planner

    # chain 결과물: Contents (contents: List[Chapter])

    return {"contents": chain.invoke(state).contents}
    # state: topic --> topic
    # Return: List[Chapter]



섹션별 내용을 처리하는 노드를 구성합니다.   
State에는 각각의 Chapter가 아닌 Chapter의 리스트인 Contents가 들어 있는데요.   

`SubState`를 이용해, 각각의 Chapter를 처리하도록 정의하겠습니다.

In [ ]:
def llm_call(state: SubState):
    # SubState :  chapter, completed_sections 2개 property

    chapter = state['chapter']

    prompt = ChatPromptTemplate([
        ('system',"아래 섹션에 대한 상세한 한국어 보고서를 작성하세요." ),
        ('user', "섹션 이름과 주제는 다음과 같습니다: {name} --> {outline}")
    ])

    chain = prompt | llm

    return {"completed_sections": [chain.invoke({'name':chapter.name, 'outline':chapter.outline}).content]}
    # 리스트로 Wrap하는 이유 중요(Reduce Operator 합치기 위해서)


# 생성된 섹션별 결과들을 결합
def synthesizer(state: State):

    completed_sections = state["completed_sections"]

    completed_report_sections = "\n\n---\n\n".join(completed_sections)
    # join: 전체 리스트 스트링으로 결합하기

    return {"final_report": completed_report_sections}




**가장 중요한 부분입니다😁😁**   
langgraph의 Send()를 이용하면, 리스트의 원소 개수만큼 서브모듈을 호출할 수 있습니다.

In [ ]:
from langgraph.types import Send

def assign_workers(state: State):
    # Send: 노드를 호출하며, 값을 전달해 준다
    # state['contents']의 개수를 기본적으로 알 수 없는데,
    # 이를 통해 개수만큼 llm_call을 생성하여 호출할 수 있음

    return [Send("llm_call", {"chapter": s}) for s in state["contents"]]

그래프를 구성합니다.

In [ ]:
builder = StateGraph(State)

builder.add_node("orchestrator", orchestrator) # 구획 짜고
builder.add_node("llm_call", llm_call) # 섹션별 글쓰고
builder.add_node("synthesizer", synthesizer) # 합치고


builder.add_edge(START, "orchestrator")

builder.add_conditional_edges("orchestrator", assign_workers, ["llm_call"])
# assign_workers의 결과에 따라 llm_call을 호출

builder.add_edge("llm_call", "synthesizer")
# 생성된 섹션들은 synthesizer로 이동

builder.add_edge("synthesizer", END) # 끝


graph = builder.compile()
graph

In [ ]:
for data in graph.stream({"topic": "GPT 1부터 최신 LLM까지의 발전과정"}, stream_mode='updates'):
    print(data)
    print('--------------')
    # 생성은 병렬적이지만 합치는 순서는 호출한 순서

In [ ]:
from IPython.display import Markdown
Markdown(data['synthesizer']["final_report"])

## 3. Evaluator-Optimizer 구조

LLM의 최초 출력을 바로 사용해도 되지만, 평가 기준을 두고 반복적으로 검증하게 한다면 그 품질을 높일 수 있습니다.    

주어진 `instruction`에 대한 파이썬 코드를 작성하고, 이를 연속적으로 최적화합니다.

In [ ]:
class State(TypedDict):
    code: str
    instruction: str
    feedback: str
    optimized: str

class Feedback(BaseModel):
    grade: Literal["optimized", "not optimized"] = Field(description="코드가 최적화되었는지 판단합니다.")
    feedback: str = Field(description="코드의 개선이 필요하다면, 어떤 부분을 개선할 수 있을지 설명합니다.")

노드 구조를 구성합니다.   
코드를 생성하는 Generator와, 코드를 비판적으로 평가하는 Optimizer 구조를 구현합니다.

In [ ]:
evaluator = llm.with_structured_output(Feedback)

def code_generator(state: State):
    # 코드를 생성하고, 피드백에 따라 리팩토링합니다.

    if state.get("feedback"):
        # 피드백이 있으면

        result = llm.invoke(
            f"""다음 문제를 해결하는 파이썬 코드를 작성하세요. 설명 없이 코드만 출력하세요.
Instruction:{state['instruction']}
다음의 피드백을 고려하세요.
Feedback: {state['feedback']}""")
    else:
        result = llm.invoke(f"""다음 문제를 해결하는 파이썬 코드를 작성하세요. 설명 없이 코드만 출력하세요.
Instruction:{state['instruction']}""")

    return {"code": result.content}


def code_evaluator(state: State):
    # 실제로는 유닛 테스트를 생성해서 검증하는 것도 필요하겠습니다.

    result = evaluator.invoke(f"""다음의 문제를 해결하는 파이썬 코드가 최적화되었는지 한국어로 평가하세요.
평가 기준은 코드의 길이와 실행 속도 및 메모리 효율성입니다.
코드의 길이가 제일 중요합니다.
---
Instruction:{state['instruction']}
---
Source Code: {state['code']}""")

    return {"optimized": result.grade, "feedback": result.feedback}




# Optimized 값에 따라 경로 설정
def route_code(state: State):
    return 'Accepted' if state["optimized"] == "optimized" else 'Rejected + Feedback'



In [ ]:
builder = StateGraph(State)

builder.add_node("code_generator", code_generator)
builder.add_node("code_evaluator", code_evaluator)

builder.add_edge(START, "code_generator")
builder.add_edge("code_generator", "code_evaluator")

builder.add_conditional_edges("code_evaluator",route_code,
                              {"Accepted": END,"Rejected + Feedback": "code_generator"})

graph = builder.compile()
graph

In [ ]:
for data in graph.stream({'instruction':'자연수보다 작은 소수 개수 구하기'}, stream_mode='updates'):
    if 'code_generator' in data:
        print(data['code_generator']['code'])
    else:
        print(data['code_evaluator']['optimized'],'-->', data['code_evaluator']['feedback'])
    print('--------------')

In [ ]:
for data in graph.stream({'instruction':'자연수 소인수분해'}, stream_mode='updates'):
    if 'code_generator' in data:
        print(data['code_generator']['code'])
    else:
        print(data['code_evaluator']['optimized'],'-->', data['code_evaluator']['feedback'])
    print('--------------')

In [ ]:
for data in graph.stream({'instruction':'Convex Hull 문제'}, stream_mode='updates'):
    if 'code_generator' in data:
        print(data['code_generator']['code'])
    else:
        print(data['code_evaluator']['optimized'],'-->', data['code_evaluator']['feedback'])
    print('--------------')